In [ ]:
%load_ext cudf.pandas

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def get_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.content,'html.parser')
    return soup,r

In [30]:
def get_anime_details(soup):
    details = {}
    details['name'] = soup.find(class_='title-name').text
    details['description'] = soup.find(itemprop='description').text
    details['score']=soup.find(itemprop='ratingValue').text
    info_list = [i.text.replace('\n','').replace(' ','') for i in 
                 soup.find(class_='leftside').find_all(class_='spaceit_pad')]

    cols = ['Type', 'Episodes','Premiered','Studios', 'Source', 
            'Rating','Members', 'Favorites']

    for col in cols:
        details[col.lower()] = next((element.split(':')[-1] for element 
                             in info_list if col in element),None)
    try:
        genres = [i[:len(i)//2] for i in next((element.split(':')[-1] for element 
                                 in info_list if 'Genres' in element),None).split(',')]
        for i in range(len(genres)):
            details[f'genre-{i+1}'] = genres[i]
    except:
        genre = next((element.split(':')[-1] for element 
                                 in info_list if 'Genre' in element),None)
        details['genre-1'] = genre[:len(genre)//2]

    return pd.DataFrame(details,index=[0])

In [4]:
def get_review_detail(review_soup):
    details = {}
    details['username'] = review_soup.find(class_='username').text.replace('\n','')
    details['anime_name'] = review_soup.find(class_='title').text
    details['anime_link'] = review_soup.find(class_='title')['href']
    details['recommended'] = review_soup.find(class_='tag').text
    details['rating_user'] = review_soup.find(class_='rating').find(class_='num').text
    return pd.DataFrame(details,index=[0])

In [ ]:
df = pd.DataFrame()
url = f'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=2&preliminary=off&spoiler=on&p={page}'

In [ ]:
pages=723
offset=1
for page in range(offset,pages+offset):
    print(f'currently on page {page}')
    soup,_ = get_soup(url)
    all_reviews = soup.find_all(class_='review-element js-review-element')
    for review in all_reviews:
        df = pd.concat([df,get_review_detail(review)],ignore_index=True)
df.to_csv(f'data/review_{offset}-{offset+pages-1}.csv',index=False)

In [5]:
df = pd.read_csv('data/review_1-722.csv')

In [11]:
anime_links = df['anime_link'].unique().tolist()

In [32]:
df = pd.DataFrame()
for anime_link in anime_links:
    soup,_ = get_soup(anime_link)
    df = pd.concat([df,get_anime_details(soup)],ignore_index=True)

In [34]:
df.to_csv('data/anime-details.csv',index=False)